## GRIB: create fieldlist from array and metadata

In [1]:
import earthkit.data
from earthkit.data import FieldList
from earthkit.data.core.temporary import temp_file
import numpy as np

earthkit.data.download_example_file("tuv_pl.grib")
ds = earthkit.data.from_source("file", "tuv_pl.grib")

In the examples below we will compute the potential temperature using the following method:

In [2]:
def potential_temperature(t, p):
    # t: temperature in K
    # p: pressure in Pa
    return t*(100000./p)**0.285611

#### Working with a single field

In this example we compute the potential temperature for the 850 hPa level using the 4th field from the fieldlist.

In [3]:
f = ds[3]
f

GribField(t,850,20180801,1200,0,0)

In [4]:
f.metadata("units")

'K'

In [5]:
f.values[:10]

array([272.53916931, 272.53916931, 272.53916931, 272.53916931,
       272.53916931, 272.53916931, 272.53916931, 272.53916931,
       272.53916931, 272.53916931])

The computations are done with numpy arrays:

In [6]:
t = f.values
print("typeOfLevel=", f.metadata("typeOfLevel"))
p = f.metadata("level")*100. #hPa -> Pa
t_new = potential_temperature(t, p)
t_new[:10]

typeOfLevel= isobaricInhPa


array([285.48786915, 285.48786915, 285.48786915, 285.48786915,
       285.48786915, 285.48786915, 285.48786915, 285.48786915,
       285.48786915, 285.48786915])

In [7]:
md_new = f.metadata().override(shortName="pt")
md_new

The original metadata remains the same.

In [8]:
print(md_new["shortName"])
print(f.metadata()["shortName"])

pt
t


In [9]:
ds_new = FieldList.from_array(t_new, md_new)
ds_new.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,pt,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


In [10]:
ds_new[0]

ArrayField()

In [11]:
ds_new[0].values[:10]

array([285.48786915, 285.48786915, 285.48786915, 285.48786915,
       285.48786915, 285.48786915, 285.48786915, 285.48786915,
       285.48786915, 285.48786915])

In [12]:
ds_new[0].metadata("name")

'Potential temperature'

In [13]:
path = "_pt_single.grib"
ds_new.save(path)
ds1 = earthkit.data.from_source("file",path)
ds1.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,pt,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll


#### Working with multiple fields

In this example we compute the potential temperature for 3 pressure levels.

In [14]:
fs = ds.sel(shortName="t", level=[850, 700, 500])
fs.ls(extra_keys=["units"])

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType,units
0,ecmf,t,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll,K
1,ecmf,t,isobaricInhPa,700,20180801,1200,0,an,0,regular_ll,K
2,ecmf,t,isobaricInhPa,500,20180801,1200,0,an,0,regular_ll,K


In [15]:
p = np.asarray(fs.metadata("level")).reshape(-1, 1)*100. # hPa -> Pa
t_new = potential_temperature(fs.values, p)
t_new.shape

(3, 84)

In [16]:
t_new[0,:10]

array([285.48786915, 285.48786915, 285.48786915, 285.48786915,
       285.48786915, 285.48786915, 285.48786915, 285.48786915,
       285.48786915, 285.48786915])

In [17]:
md_new = [f.metadata().override(shortName="pt") for f in fs]
ds_new = FieldList.from_array(t_new, md_new)
ds_new.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,pt,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
1,ecmf,pt,isobaricInhPa,700,20180801,1200,0,an,0,regular_ll
2,ecmf,pt,isobaricInhPa,500,20180801,1200,0,an,0,regular_ll


In [18]:
ds_new[0].values[:10]

array([285.48786915, 285.48786915, 285.48786915, 285.48786915,
       285.48786915, 285.48786915, 285.48786915, 285.48786915,
       285.48786915, 285.48786915])

In [19]:
ds_new.metadata("name")

['Potential temperature', 'Potential temperature', 'Potential temperature']

In [20]:
path = "_pt_multi.grib"
ds_new.save(path)
ds1 = earthkit.data.from_source("file", path)
ds1.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,pt,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
1,ecmf,pt,isobaricInhPa,700,20180801,1200,0,an,0,regular_ll
2,ecmf,pt,isobaricInhPa,500,20180801,1200,0,an,0,regular_ll


#### Performing the computations in a loop

In [21]:
fs = ds.sel(shortName="t", level=[850, 700, 500])

# create an empty fieldlist
ds_r = FieldList()

for f in fs:
    p = f.metadata("level")*100. # hPa -> Pa
    t_new = potential_temperature(f.values, p)
    md_new = f.metadata().override(shortName="pt")
    
    # create new numpy fieldlist with a single field
    ds_new = FieldList.from_array(t_new, md_new)

    # add it to the resulting fieldlist
    ds_r += ds_new

In [22]:
ds_r.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,pt,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
1,ecmf,pt,isobaricInhPa,700,20180801,1200,0,an,0,regular_ll
2,ecmf,pt,isobaricInhPa,500,20180801,1200,0,an,0,regular_ll


In [23]:
path = "_pt_from_loop.grib"
ds_r.save(path)
ds1 = earthkit.data.from_source("file", path)
ds1.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,pt,isobaricInhPa,850,20180801,1200,0,an,0,regular_ll
1,ecmf,pt,isobaricInhPa,700,20180801,1200,0,an,0,regular_ll
2,ecmf,pt,isobaricInhPa,500,20180801,1200,0,an,0,regular_ll
